In [6]:
import json
import requests

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

from langchain.agents import create_agent
from dotenv import load_dotenv
load_dotenv()


True

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

prompt = """
Analiza el siguiente problema y responde con:
- un resumen
- los riesgos principales
- un nivel de confianza

Problema:
La empresa depende de un único proveedor de infraestructura en la nube.
"""

response = llm.invoke(prompt)
print(response.content)

**Resumen:**
La empresa está en una situación de dependencia crítica al contar con un solo proveedor de infraestructura en la nube. Esto significa que todos sus servicios y operaciones digitales están vinculados a la disponibilidad, precios y políticas de este proveedor específico. Cualquier cambio o problema con el proveedor podría impactar significativamente en las operaciones de la empresa.

**Riesgos principales:**
1. **Riesgo de interrupción del servicio:** Si el proveedor experimenta una caída o interrupción, la empresa podría enfrentar tiempos de inactividad que afecten su operación y servicio al cliente.
2. **Riesgo financiero:** Cambios en la estructura de precios del proveedor podrían aumentar los costos operativos de la empresa de manera inesperada.
3. **Riesgo de seguridad:** La empresa está expuesta a las políticas de seguridad del proveedor. Cualquier vulnerabilidad o brecha de seguridad en el proveedor podría comprometer los datos y operaciones de la empresa.
4. **Riesgo

In [8]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
# -------------------------
# Modelo Pydantic (contrato)
# -------------------------
class ConfidenceLevel(str, Enum):
    low = "low"
    medium = "medium"
    high = "high"


class AnalysisResult(BaseModel):
    summary: str = Field(description="Resumen claro del problema")
    risks: List[str] = Field(description="Lista de riesgos identificados")
    confidence: ConfidenceLevel


# -------------------------
# Parser
# -------------------------
parser = PydanticOutputParser(pydantic_object=AnalysisResult)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un analista de riesgos.\n"
            "Debes responder EXCLUSIVAMENTE en el formato indicado.\n"
            "{format_instructions}",
        ),
        ("human", "{problem}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI(model="gpt-4o", temperature=0)

chain = prompt | llm | parser


# -------------------------
# Ejecución
# -------------------------
result = chain.invoke(
    {
        "problem": "La empresa depende de un único proveedor de infraestructura en la nube."
    }
)

print(result,'\n')
print('Summary: ',result.summary)
print('Risks: ')
for risk in result.risks:
    print('-', risk)
print('Confidence: ', result.confidence)


summary='La empresa depende de un único proveedor de infraestructura en la nube, lo que puede generar riesgos significativos en caso de fallos o problemas con dicho proveedor.' risks=['Interrupciones del servicio si el proveedor experimenta problemas técnicos.', 'Dependencia excesiva que puede limitar la flexibilidad y negociación de precios.', 'Riesgo de seguridad si el proveedor sufre una brecha de datos.', 'Dificultades para cambiar de proveedor en el futuro debido a la falta de diversificación.', 'Posibles aumentos de costos si el proveedor decide cambiar su estructura de precios.'] confidence=<ConfidenceLevel.high: 'high'> 

Summary:  La empresa depende de un único proveedor de infraestructura en la nube, lo que puede generar riesgos significativos en caso de fallos o problemas con dicho proveedor.
Risks: 
- Interrupciones del servicio si el proveedor experimenta problemas técnicos.
- Dependencia excesiva que puede limitar la flexibilidad y negociación de precios.
- Riesgo de segu